<a href="https://colab.research.google.com/github/sahil301290/rapids_ML_DS/blob/main/Unit_4_NLP_12_Spell_Corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from collections import Counter

In [2]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [3]:
# create a frequency table of all the words of the document
all_words = Counter(words(open('sample_spell.txt').read()))

In [4]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [5]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [6]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [7]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [8]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

In [9]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

In [10]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [11]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

336
{'monnev', 'mohnney', 'mbnney', 'monneys', 'moncney', 'mwnney', 'monhney', 'moynney', 'moenney', 'uonney', 'monnuey', 'minney', 'monbney', 'mmnney', 'dmonney', 'monxey', 'mrnney', 'mojnney', 'monnoey', 'monnewy', 'monnezy', 'money', 'monnej', 'monnel', 'molnney', 'yonney', 'mondey', 'mjonney', 'mouney', 'mononey', 'mohney', 'monnvy', 'moniney', 'monaey', 'monnef', 'ponney', 'monnby', 'monnfy', 'monneyz', 'monnsy', 'monnjey', 'mnonney', 'monnemy', 'monneay', 'mobnney', 'wmonney', 'mtonney', 'hmonney', 'monley', 'mconney', 'monneg', 'moanney', 'monmney', 'monnfey', 'monneyt', 'monneya', 'mocney', 'monneiy', 'monnry', 'monnay', 'monnety', 'monnez', 'motney', 'monneyj', 'monkey', 'monnery', 'monneyu', 'msonney', 'monnoy', 'mongney', 'monzey', 'monnkey', 'mdonney', 'monneyb', 'mqonney', 'moeney', 'monnepy', 'monnem', 'gonney', 'monaney', 'monneey', 'moneey', 'monniy', 'monfney', 'amonney', 'mojney', 'monrney', 'monnvey', 'movnney', 'myonney', 'monnegy', 'mknney', 'monnejy', 'monneyx', '

In [12]:
print(known(edits_one("monney")))

{'money', 'monkey'}


In [13]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

51013
{'money', 'monkey'}


In [14]:
# Let's look at possible corrections of a word
print(possible_corrections("monney"))

{'money', 'monkey'}


In [15]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [16]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [17]:
# test spell check
print(spell_check("monney"))

Did you mean money?
